# Import Necessary Modules

In [1]:
import nltk
import random
from nltk.classify.scikitlearn import SklearnClassifier

from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC, NuSVC

from nltk.classify import ClassifierI
from statistics import mode

from nltk.tokenize import word_tokenize

# Generate Vote Classifier

In [2]:
class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers

    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)

    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)

        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf

# Read Data

In [3]:
movie_pos = open("movie_reviews/positive.txt","r").read()
movie_neg = open("movie_reviews/negative.txt","r").read()

# Implementation

In [4]:
documents = []

for r in movie_pos.split('\n'):
    documents.append( (r, "pos") )

for r in movie_neg.split('\n'):
    documents.append( (r, "neg") )

In [5]:
all_words = []

short_pos_words = word_tokenize(movie_pos)
short_neg_words = word_tokenize(movie_neg)

for w in short_pos_words:
    all_words.append(w.lower())

for w in short_neg_words:
    all_words.append(w.lower())

all_words = nltk.FreqDist(all_words)

In [6]:
word_features = list(all_words.keys())[:5000]

def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

featuresets = [(find_features(rev), category) for (rev, category) in documents]

In [7]:
random.shuffle(featuresets)

In [8]:
training_set = featuresets[:10000]
testing_set =  featuresets[10000:]

In [9]:
classifier = nltk.NaiveBayesClassifier.train(training_set)
print("Original Naive Bayes Algo accuracy percent:", (nltk.classify.accuracy(classifier, testing_set))*100)
classifier.show_most_informative_features(15)

MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print("MNB_classifier accuracy percent:", (nltk.classify.accuracy(MNB_classifier, testing_set))*100)

BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_set)
print("BernoulliNB_classifier accuracy percent:", (nltk.classify.accuracy(BernoulliNB_classifier, testing_set))*100)

LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogisticRegression_classifier, testing_set))*100)

SGDClassifier_classifier = SklearnClassifier(SGDClassifier())
SGDClassifier_classifier.train(training_set)
print("SGDClassifier_classifier accuracy percent:", (nltk.classify.accuracy(SGDClassifier_classifier, testing_set))*100)

LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
print("LinearSVC_classifier accuracy percent:", (nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100)

NuSVC_classifier = SklearnClassifier(NuSVC())
NuSVC_classifier.train(training_set)
print("NuSVC_classifier accuracy percent:", (nltk.classify.accuracy(NuSVC_classifier, testing_set))*100)


voted_classifier = VoteClassifier(
                                  NuSVC_classifier,
                                  LinearSVC_classifier,
                                  MNB_classifier,
                                  BernoulliNB_classifier,
                                  LogisticRegression_classifier)

print("voted_classifier accuracy percent:", (nltk.classify.accuracy(voted_classifier, testing_set))*100)

Original Naive Bayes Algo accuracy percent: 75.75301204819277
Most Informative Features
              engrossing = True              pos : neg    =     21.6 : 1.0
              refreshing = True              pos : neg    =     14.3 : 1.0
               inventive = True              pos : neg    =     14.3 : 1.0
            refreshingly = True              pos : neg    =     12.3 : 1.0
                    warm = True              pos : neg    =     11.7 : 1.0
                   vivid = True              pos : neg    =     11.6 : 1.0
               wonderful = True              pos : neg    =     11.3 : 1.0
                provides = True              pos : neg    =     11.3 : 1.0
               realistic = True              pos : neg    =     10.9 : 1.0
             mesmerizing = True              pos : neg    =     10.3 : 1.0
                  stupid = True              neg : pos    =     10.2 : 1.0
                powerful = True              pos : neg    =     10.1 : 1.0
            

E:\Anaconda\Aditya\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier_classifier accuracy percent: 72.28915662650603
LinearSVC_classifier accuracy percent: 73.64457831325302
NuSVC_classifier accuracy percent: 75.6024096385542
voted_classifier accuracy percent: 75.75301204819277
